In [1]:
from bs4 import BeautifulSoup
import requests
import html5lib
import re
import pandas as pd
import numpy as np
# pip install pyopenssl ndg-httpsclient pyasn1

In [2]:
cookies = {
    'ccsid': '668-8707266-8901325',
    '__qca': 'P0-1002348396-1661184067604',
    'p': 'ZqHh-ArV88dYkVHfgapFDgVJRXU5lLzNc4VfzdNPfI5ZMVgd',
    'likely_has_account': 'true',
    'srb_8': '0_wl',
    'locale': 'en',
    'u': 'oTJu21ixofG3HvrbsDKN1hg8e8FfKW_FUT1R8ZnFKULdyBce',
    'csm-sid': '696-1026276-2871101',
    'logged_out_browsing_page_count': '2',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'en-US,en;q=0.9,id;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    # 'Cookie': 'ccsid=668-8707266-8901325; __qca=P0-1002348396-1661184067604; p=ZqHh-ArV88dYkVHfgapFDgVJRXU5lLzNc4VfzdNPfI5ZMVgd; likely_has_account=true; srb_8=0_wl; locale=en; u=oTJu21ixofG3HvrbsDKN1hg8e8FfKW_FUT1R8ZnFKULdyBce; csm-sid=696-1026276-2871101; logged_out_browsing_page_count=2',
    'If-None-Match': 'W/"ac56aa1a7ffc3395f615ff303dbf9c1d"',
    'Referer': 'https://www.goodreads.com/list/show/23974.Worst_Rated_Books_on_Goodreads?page=6',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Not?A_Brand";v="8", "Chromium";v="108", "Google Chrome";v="108"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}

In [3]:
cookies2 = {
    'ccsid': '668-8707266-8901325',
    '__qca': 'P0-1002348396-1661184067604',
    'p': 'ZqHh-ArV88dYkVHfgapFDgVJRXU5lLzNc4VfzdNPfI5ZMVgd',
    'likely_has_account': 'true',
    'srb_8': '0_wl',
    'locale': 'en',
    'u': 'oTJu21ixofG3HvrbsDKN1hg8e8FfKW_FUT1R8ZnFKULdyBce',
    'csm-sid': '696-1026276-2871101',
    'logged_out_browsing_page_count': '2',
    '_session_id2': '32ae2b1ba7e2e21baf4eae9927378a3f',
}

headers2 = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'en-US,en;q=0.9,id;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    # 'Cookie': 'ccsid=668-8707266-8901325; __qca=P0-1002348396-1661184067604; p=ZqHh-ArV88dYkVHfgapFDgVJRXU5lLzNc4VfzdNPfI5ZMVgd; likely_has_account=true; srb_8=0_wl; locale=en; u=oTJu21ixofG3HvrbsDKN1hg8e8FfKW_FUT1R8ZnFKULdyBce; csm-sid=696-1026276-2871101; logged_out_browsing_page_count=2; _session_id2=32ae2b1ba7e2e21baf4eae9927378a3f',
    'If-None-Match': 'W/"cd6780e9369edb82b27b62763331a7b5"',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Not?A_Brand";v="8", "Chromium";v="108", "Google Chrome";v="108"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}

In [4]:
def clean_publish_year(raw_year):
    if type(raw_year) is None:
        return "Missing"
    result = re.search(r"(\b\d+)$", raw_year)
    if result == None:
        return 0
    else:
        return result.group(1)

def clean_rating(raw_rating):
    if type(raw_rating) is None:
        return 0
    result = re.search(r"(\b\d+.\b\d+)", raw_rating)
    if result == None:
        return 0
    else:
        return result.group(1)

def clean_raters(raw_raters):
    if type(raw_raters) is None:
        return 0
    result = re.search(r"((?:\d+,?\d+)*) ratings", raw_raters)
    if result == None:
        return 0
    else:
        return result.group(1)
    
def clean_genres(raw_genres):
    genreList = []
    if raw_genres == None:
        return "Missing"
    else:
        for genre in raw_genres:
            if  genre == None:
                continue
            else:
                genreList.append(genre.find("a").get_text())
        return genreList
# def clean_readers(raw_readers):
#     result = re.search(r"(\b\d+)", raw_readers)
#     if result == None:
#         return "None"
#     else:
#         return result.group(1)

def solve_nonetype_error(value):
    if value is not None:
        return value.text.strip()
    else:
        return "Missing"



In [7]:

books = []
for page in range(1, 7):
    response = requests.get("https://www.goodreads.com/list/show/23974.Worst_Rated_Books_on_Goodreads?page=" + str(page), cookies = cookies, headers=headers)
    html_soup = BeautifulSoup(response.text, 'html5lib')
    book_container = html_soup.find_all('table', class_ = "tableList js-dataTooltip")
    print("this is page " , page)
    for book in book_container:
        bookTitle = solve_nonetype_error(book.find('a', class_ = "bookTitle"))
       
        bookAuthor =solve_nonetype_error(book.find('a', class_ = "authorName"))

        # After(use comments)
        # year = clean_publish_year(solve_nonetype_error(book.find('span', class_ = "greyText smallText")))
        rating = clean_rating(solve_nonetype_error(book.find('span', class_ = "greyText smallText")))
        raters = str(clean_raters(solve_nonetype_error(book.find('span', class_ = "greyText smallText")))).replace(",", "")
        int(raters)
        # # readerNum = clean_readers(book.find('span', class_ = "greyText statistic").text.strip())
        
        # Before (use comments)
        # year = solve_nonetype_error(book.find('span', class_ = "greyText smallText"))
        # rating = solve_nonetype_error(book.find('span', class_ = "minirating"))
        # raters = solve_nonetype_error(book.find('span', class_ = "minirating"))
        # readerNum = book.find('span', class_ = "greyText statistic").text.strip()
        print(bookTitle)
        link = book.a['href']
        completeLink = "https://www.goodreads.com" + link
        # getGenre = requests.get(completeLink)
        getGenre = requests.get(completeLink,  cookies = cookies2, headers=headers2)
        genreParsing = BeautifulSoup(getGenre.text, 'html5lib')
        genres = clean_genres(genreParsing.findAll("div", class_=  "elementList", style = None))
        genreStr = ",".join(genres)
        book_data = [bookTitle, bookAuthor, rating, raters, genreStr]
        books.append(book_data)
books = pd.DataFrame(books, columns=["Title", "Author", "Rating", "Raters", "Genres"])
books.head()

this is page  1
Besos entre líneas
this is page  2
Kind of a Big Deal
this is page  3
I Wish My Kids Had Cancer: A Family Surviving the Autism Epidemic
this is page  4
Mystification
this is page  5
Red Skull
this is page  6
The Astor Orphan: A Memoir


,Title,Author,Rating,Raters,Genres
0,Besos entre líneas,May R. Ayamonte,"2.15 avg rating — 2,183 ratings","2.15 avg rating — 2,183 ratings","Young Adult,Romance,Contemporary,Romance"
1,Kind of a Big Deal,Shannon Hale,"2.83 avg rating — 3,331 ratings","2.83 avg rating — 3,331 ratings","Young Adult,Fantasy,Contemporary,Fiction,Roman..."
2,I Wish My Kids Had Cancer: A Family Surviving ...,Michael Alan,1.26 avg rating — 35 ratings,1.26 avg rating — 35 ratings,
3,Mystification,Edgar Allan Poe,2.60 avg rating — 221 ratings,2.60 avg rating — 221 ratings,"Short Stories,Classics,Literature,Horror,Ficti..."
4,Red Skull,Joshua Williamson,2.84 avg rating — 212 ratings,2.84 avg rating — 212 ratings,"Sequential Art,Superheroes,Sequential Art,Comi..."


In [8]:
books.to_csv("WorstRatedBooks.csv", index = False)


In [12]:
df = pd.read_csv("WorstRatedBooksCleaned.csv")

df = df[df.Author != "Missing"]


df = df.drop("Unnamed: 0", axis =1)
df = df.dropna()

df.isnull().sum()

df.tail()

df.to_csv("WorstBooks.csv", index = False)




In [30]:
# test = pd.read_csv("WorstBooks.csv")

test2 = pd.read_csv("ToReadCleaned.csv")

test2 = test2.drop("Year", axis = 1)

test2.head()

test2.to_csv("ToReadBooks.csv", index = False)



In [31]:
test = pd.read_csv("WorstBooks.csv")
test.head()

,Title,Author,Rating,Raters,Genres
0,Besos entre líneas,May R. Ayamonte,2.15,2183,"Young Adult,Romance,Contemporary,Romance"
1,Birth Control Is Sinful in the Christian Marri...,Eliyzabeth Yanne Strong-Anderson,1.86,273,"Religion,Nonfiction"
2,A Parent's Guide to Preventing Homosexuality,Joseph Nicolosi,1.74,708,"Parenting,Psychology,Horror,Sexuality,Nonficti..."
3,Coming Out Straight: Understanding and Healing...,Richard Cohen,1.86,564,"Psychology,Fiction,Social Issues"
4,Dianetics: The Modern Science of Mental Health,L. Ron Hubbard,2.42,3205,"Religion,Nonfiction,Philosophy,Psychology,Self..."


In [34]:
test = pd.read_csv("WorstBooks.csv")

test2 = pd.read_csv("ToReadBooks.csv")



df_append = test.append(test2, ignore_index=True)


df_append = df_append.dropna()

df_append.isnull().any()
df_append = df_append.sample(frac = 1)
df_append.to_csv("CombinedBooks.csv", index = False)


C:\Users\10\AppData\Local\Temp\ipykernel_9556\1971954267.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_append = test.append(test2, ignore_index=True)


,Title,Author,Rating,Raters,Genres
1306,"Book of Night (Book of Night, #1)",Holly Black,3.55,34211,"Fantasy,Fantasy,Adult,Fiction,Fantasy,Mystery,..."
828,The Ballad of Songbirds and Snakes (The Hunger...,Suzanne Collins,3.83,327719,"Young Adult,Science Fiction,Fiction,Fantasy,Sc..."
49,Michelle Remembers,Michelle Smith,2.65,625,"Nonfiction,Horror,Crime,Biography,Autobiograph..."
1458,The Huntress (Kindle Edition),Kate Quinn,4.29,129999,"Historical,Fiction,Historical,War,Audiobook,Wa..."
1093,Landline (Hardcover),Rainbow Rowell,3.54,162463,"Fiction,Romance,Contemporary,Adult,Womens Fict..."
